<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
تشخیص اخبار دروغین
</font>
</h1>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
مقدمه و صورت مسئله
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
مطمئنا بارها شنیده‌اید که در فضای مجازی اخبار دروغ و شایعه بسیار رایج است. افرادی هستند که به هر دلیل اقدام به انتشار خبرهای دروغ می‌کنند. برخی این دروغ‌ها از جنس خبرهای سیاسی، اقتصادی یا حواشی راجع به زندگی افراد مشهور و سلبریتی‌هاست. اما به هر نحوی که باشد، دروغ رفتار خوبی نیست.
    <br>
    پس به عنوان یک مخاطب خوب است که بتوانیم تشخیص دهیم چه خبری حقیقت و چه خبری دروغ است. اگر از دید یک خبرگزاری هم به ماجرا نگاه کنیم، متوجه خواهیم شد که اصلا دوست نداریم یک خبر دروغ اقتصادی یا سیاسی از زبان ما منتشر شود؛ در غیر این صورت عواقب پیشبینی‌نشده‌ای در انتظارمان خواهد بود!
    <br>
    پس حال که به اهمیت و کاربرد تشخیص اخبار دروغین پی برده‌ایم، خوب است خودمان تلاش کنیم مدلی آموزش دهیم که خبرهای دروغ را از حقیقت جدا کنیم.
    <br>
    البته تشخیص این امر، وابسته به توانایی پردازش زبان طبیعی است و با دانش یادگیری ماشین کلاسیک، به سادگی نمی‌توان مسئله رو حل کرد. برای اینکه مسئله قابل حل باشد، بخش «پردازش زبان طبیعی» را خود ما انجام داده‌ایم و نتیجه آن‌ را در قالب بخشی از مجموعه‌داده در اختیار شما قرار داده‌ایم.
</font>
</p>

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
وارد کردن کتابخانه‌های مورد نیاز
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    ابتدا کتابخانه‌های مورد نیازتان را وارد کنید.
</font>
</p>

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as ss
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, f1_score

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
معرفی مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
مجموعه‌داده این تمرین، مانند تمرین‌های قبلی به دو مجموعه آموزش و آزمون تقسیم‌بندی می‌شود. اما نکته حائز اهمیت این است که هر کدام از این دو مجموعه آزمون و آموزش، خود دارای دو مجموعه داده است. به عبارت بهتر مجموعه آموزش دارای دو مجموعه‌داده به اسم <code>news_train.csv</code> و <code>news_train_text_vectors.npz</code> است (مجموعه آزمون هم همینطور است).
    <br>
    در فایل <code>news_train.csv</code> اطلاعات کلی مانند زمان انتشار، منبع و برچسب خبر آمده است؛ اما خود خبر را در این مجموعه‌داده نداریم. بلکه خود خبر را می‌توان در فایل <code>news_train_text_vectors.npz</code> جستجو کرد. اما در <code>news_train_text_vectors.npz</code> متن خبر قرار نگرفته است، بلکه به کمک روش‌های پردازش زبان طبیعی، متن هر خبر را به یک بردار ۴۲۱۴۱ المانی تبدیل کرده‌ایم و به ازای هر خبر، یک بردار با ابعاد <code>(42141, 1)</code> وجود دارد.
    <br>
    توضیحات مجموعه‌داده <code>news_train.csv</code> را در جدول زیر می‌توانید مشاهده کنید.
</font>
</p>

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|author|شخصی که متن خبر را نوشته است|
|published|تاریخ و ساعتی که خبر منتشر شده است|
|site_url|سایت منتشرکننده خبر|
|type|برای هر خبر یک دسته یا یک نوعی نیز ذخیره کرده‌ایم. این ستون می‌تواند یکی از مقادیر `bs` به معنی *پرت* یا ‍‍‍‍`conspiracy` به معنی *توطئه آمیز* یا `bias` به معنی *سوگیری یا متعصبانه* یا `hate` به معنی *نفرت‌افکن یا خشم‌آلود* یا ‍`satrie` به معنی *طنز* یا `junksci` به معنی *به درد نخور یا آشغال* و `fake` به معنی *دروغ* باشد|
|label|برچسب خبر را نشان می‌دهد. این ستون یا `Fake` است یا `Real`|
    
</font>
</div>
</center>


<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    در کنار فایل <code>news_train.csv</code> فایل <code>news_train_text_vectors.npz</code> نیز قرار دارد. کلیات این فایل را در قسمت قبلی توضیح دادیم. به حجیم بودن فایل و صرفه‌جویی درفضای ذخیره‌سازی و کاهش زمان برای بارگذاری فایل، به جای فرمت <code>csv</code> آن را در فرمت <code>npz</code> در اختیار شما قرار دادیم. اما پیشنهاد می‌کنیم برای راحتی خودتان، پس از خواندن فایل، آن را به یک دیتافریم با ۱۵۰۰ سطر و ۴۲۱۴۱ ستون تبدیل کنید.
    <br>
    اینطور در نظر داشته باشید که متن هر خبر را با ۴۲۱۴۱ ویژگی برایتان توصیف کرده‌ایم!
</font>
</p>


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
خواندن مجموعه داده
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    در ابتدا نیاز است فایل‌های مجموعه‌داده را بخوانید. نمونه‌های آموزشی در فایل <code>news_train.csv</code> و <code>news_train_text_vectors.npz</code> و نمونه‌های آزمون که باید دسته‌ی آن‌ها را پیش‌بینی کنید در فایل <code>news_test.csv</code> و <code>news_test_text_vectors.npz</code> ذخیره شده‌اند. اگر لازم دانستید می‌توانید به دلخواه خود بخشی از دادگان آموزشی را به عنوان دادگان اعتبارسنجی نیز جدا کنید.
</font>
</p>

In [2]:
train_data = pd.read_csv('../data/news_train.csv')
train_data_text_vectors = ss.load_npz('../data/news_train_text_vectors.npz')

test_data = pd.read_csv('../data/news_test.csv')
test_data_text_vectors = ss.load_npz('../data/news_test_text_vectors.npz')

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
پیش‌پردازش و مهندسی ویژگی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    در این سوال شما می‌توانید از هر تکنیک پیش‌پردازش/مهندسی ویژگی که در فصل‌های گذشته آموختید، استفاده کنید.
    <br>
    تکنیک‌هایی که استفاده می‌کنید به شکل مستقیم مورد ارزیابی توسط سامانه داوری قرار <b>نمی‌گیرند.</b> بلکه همه آن‌ها در دقت مدل شما تاثیر خواهند گذاشت؛ بنابراین هر چه پیش‌پردازش/مهندسی ویژگی بهتری انجام دهید تا دقت مدل بهبود پیدا کند، امتیاز بیشتری از این سوال کسب خواهید کرد.
    <br>
    از آنجایی که قرار است در این تمرین از الگوریتم ماشین بردار پشتیبان استفاده کنید، حتما باید مجموعه‌داده <code>news_train_text_vectors.npz</code> را با روش‌های کاهش ابعادی که یاد گرفته‌اید، به نحوی تغییر دهید که در زمان معقولی توسط الگوریتم <code>svm</code> پردازش شود. اگر چه الگوریتمی که برای کاهش ابعاد استفاده می‌کنید در کوئرا بررسی نمی‌شود، اما پیشنهاد ما استفاده از <code>PCA</code> است. آنچه اهمیت دارد آن است که بین کوچک‌کردن داده و کیفیت مدل تعادل برقرار کنید. طبیعی است که هرچقدر داده کوچک‌تر باشد، احتمالا الگوریتم ماشین بردار پشتیبان زودتر به نتیجه می‌رسد؛ اما در نظر داشته باشید با کاهش بعد، شما عملا بخشی از داده را از دست می‌دهید!
</font>
</p>

<h3 align=right style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
    استفاده از <code>scikit-learn</code>
</font>
</h3>


<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    الگوریتم <i>تحلیل مولفه‌های اصلی</i> با نام <code>PCA</code> در پکیج <code>decomposition</code> این کتابخانه در دسترس است. برخی از آرگومان‌های مهم آن در جدول زیر آمده است، اما جهت مطالعه‌ی کامل‌تر مستندات می‌توانید به <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html" target="_blank">این لینک</a> مراجعه فرمایید.
    <br>
    برای استفاده از این الگوریتم، می‌توانید با اجرای <code>from sklearn.decomposition import PCA</code> آن را وارد (<code>import</code>) کنید.
    <br>
    
</font>
</p>


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>

|آرگومان|جنس و تایپ|توضیحات|
|:------:|:--------:|:---:|
|n_components|<code>int</code> یا <code>float</code>|تعداد مولفه‌هایی است که قصد داریم پس از اجرای الگوریتم، آن‌ها را داشته باشیم و حذف نشوند. اگر به این آرگومان مقدار ندهیم، همه مولفه‌ها حفظ خواهند شد. اگر جنس این آرگومان <code>int</code> باشد، نشانگر تعداد مولفه‌هایی است که قصد داریم حفظ شود. اما اگر از جنس <code>float</code> و به شکل $0< n\_components< 1$ باشد، به این معنا خواهد بود که به اندازه‌ای از مولفه‌ها حفظ شوند که واریانس داده‌ها بزرگتر از `n_componets`٪ باشد. بنابراین می‌توان اینگونه برداشت کرد که هرچه این عدد به ۱ نزدیک‌تر باشد، چون قرار است داده واریانس زیادی داشته باشد، پس مولفه‌های بیشتری حفظ می‌شوند.|
    
</font>
</div>
</center>



<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    کلاس <code>PCA</code> دارای سه متد به شرح زیر است: 
</font>
</p>


<center>
<div dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|اسم متد|توضیحات|
|:------:|:---:|
|fit(X)|الگوریتم را با X که در ابعاد `(n_samples, n_features)` است|
|transform(X)|این تابع X که از ابعاد `(n_samples, n_features)` است را به یک ماتریس دیگری با ابعاد `(n_samples, n_components)` تبدیل می‌کند. پس خروجی این تابع یک ماتریس به ابعاد `(n_samples, n_components)` خواهد بود|
|fit_transform(X)|ابتدا تابع <code>fit</code> و سپس تابع <code>transform</code> را اجرا می‌کند|
</font>
</div>
</center>

In [4]:
N_COMPONENTS = 200



svd = TruncatedSVD(n_components=N_COMPONENTS, random_state=42)
X_train_text = svd.fit_transform(train_data_text_vectors) 
X_test_text  = svd.transform(test_data_text_vectors)    

explained_var = svd.explained_variance_ratio_.sum()

def engineer_metadata_features(df, fit_encoders=True, encoders=None):
    """Extract numeric features from the metadata CSV."""
    features = pd.DataFrame(index=df.index)
    
    type_map = {'fake': 0, 'bs': 1, 'conspiracy': 2, 'bias': 3,
                'hate': 4, 'satire': 5, 'junksci': 6}
    features['type_enc'] = df['type'].map(type_map).fillna(-1)
    
    for t, code in type_map.items():
        features[f'type_{t}'] = (df['type'] == t).astype(int)
    features['type_unknown'] = df['type'].isna().astype(int)
    
    if fit_encoders:
        top_sites = df['site_url'].value_counts().head(50).index.tolist()
        encoders = {'top_sites': top_sites}
    
    top_sites = encoders['top_sites']
    site_url_clean = df['site_url'].fillna('unknown')
    features['site_url_enc'] = site_url_clean.apply(
        lambda x: top_sites.index(x) if x in top_sites else len(top_sites)
    )
    
    features['has_author'] = df['author'].notna().astype(int)
    
    pub = pd.to_datetime(df['published'], errors='coerce', utc=True)
    
    if pub.notna().sum() > 0 and hasattr(pub, 'dt'):
        try:
            features['pub_hour']      = pub.dt.hour.fillna(-1)
            features['pub_dayofweek'] = pub.dt.dayofweek.fillna(-1)
            features['pub_month']     = pub.dt.month.fillna(-1)
        except AttributeError:
            features['pub_hour']      = -1
            features['pub_dayofweek'] = -1
            features['pub_month']     = -1
    else:

        try:
            pub_numeric = pd.to_numeric(df['published'], errors='coerce')
            pub_dt = pd.to_datetime(pub_numeric, unit='s', errors='coerce')
            features['pub_hour']      = pub_dt.dt.hour.fillna(-1)
            features['pub_dayofweek'] = pub_dt.dt.dayofweek.fillna(-1)
            features['pub_month']     = pub_dt.dt.month.fillna(-1)
        except Exception:
            features['pub_hour']      = -1
            features['pub_dayofweek'] = -1
            features['pub_month']     = -1
    
    features['has_published'] = df['published'].notna().astype(int)
    
    return features.values.astype(float), encoders


X_train_meta, encoders = engineer_metadata_features(train_data, fit_encoders=True)
X_test_meta, _         = engineer_metadata_features(test_data, fit_encoders=False, encoders=encoders)

X_train_combined = np.hstack([X_train_text, X_train_meta])
X_test_combined  = np.hstack([X_test_text,  X_test_meta])

y_train = train_data['label'].values

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_combined)  # fit on train only
X_test_scaled  = scaler.transform(X_test_combined)

X_train, X_val, y_train_split, y_val = train_test_split(
    X_train_scaled, y_train,
    test_size=0.15, random_state=42, stratify=y_train
)


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
مدل‌سازی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    حال که داده را پاکسازی کرده و احتمالا ویژگی‌هایی را به آن افزوده یا از آن حذف کرده‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
    <br>
    <b>توجه داشته باشید که برای پیشبینی متغیر هدف این مسئله حتما باید از الگوریتم <code>SVM</code> استفاده کنید. در غیر این صورت، نمره‌ای دریافت نخواهید کرد.</b>
    <br>
    نحوه استفاده از این الگوریتم را در تمرین‌ها و درسنامه‌های قبلی آموخته‌اید.
</font>
</p>

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
آموزش مدل
</font>
</h3>

In [5]:
model = SVC(
    kernel='rbf',
    C=10,
    gamma='scale',
    probability=False,
    random_state=42,
)
model.fit(X_train, y_train_split)

SVC(C=10, random_state=42)

<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>f1_score</code> نام دارد.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی دادگان آموزش یا اعتبارسنجی ارزیابی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazirmatn" size=3>
 جهت کسب امتیاز کامل نیاز است تا پاسخ شما حداقل امتیاز <code>80</code> را با توجه به معیار معرفی‌شده کسب نماید.
</font>
</p>

In [6]:
y_pred_val = model.predict(X_val)

f1 = f1_score(y_val, y_pred_val, average='weighted')
print(f'Validation Weighted F1 Score: {f1:.4f} ({f1*100:.2f}%)')
print(classification_report(y_val, y_pred_val))

Validation Weighted F1 Score: 0.9955 (99.55%)
              precision    recall  f1-score   support

        Fake       0.99      1.00      1.00       148
        Real       1.00      0.99      0.99        77

    accuracy                           1.00       225
   macro avg       1.00      0.99      1.00       225
weighted avg       1.00      1.00      1.00       225



<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
 پیش‌بینی برای داده تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    پس از مهندسی ویژگی و مدلسازی، الگوریتمی دارید که می‌تواند شما را از متغیرهای مستقل به متغیر هدف برساند.
    <br>
    از این مدل برای پیش‌بینی نمونه‌های موجود در داده تست استفاده کنید و نتایج را در قالب جدول (<code>dataframe</code>) زیر آماده کنید.
</font>
</p>

<div dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|------|---|
|label|برچسب پیشبینی شده|
    
</font>
</div>



<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    اسم دیتافریم باید <i>submission</i> باشد؛ در غیر این صورت، سامانه داوری نمی‌تواند تلاش‌ شما را ارزیابی کند.
    <br>
    این دیتافریم تنها شامل ۱ ستون با اسم <i>label</i> است و ۳۴۶ سطر دارد.
    <br>
    به ازای هر سطر موجود در دیتافریم <i>test</i> شما باید یک مقدار پیشبینی شده داشته باشید.
    <br>
    جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته در جواب شما، اعداد ستون <i>label</i> ممکن است متفاوت باشد.
</font>
</p>

<div style="text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
||label|
|----|-----|
|0|Fake|
|1|Fake|
|2|Real|
|3|Fake|
|4|Real|
</font>
</div>



In [7]:
y_pred_test = model.predict(X_test_scaled)
submission = pd.DataFrame({'label': y_pred_test})
print(submission.head())

  label
0  Real
1  Fake
2  Real
3  Real
4  Real


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) تا در صورت نیاز به پشتیبانی امکان بررسی کد شما وجود داشته باشد.
</font>
</p>

In [8]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

joblib.dump(model, 'model')
submission.to_csv('submission.csv', index=False)
file_names = ['fake_news_detection.ipynb', 'submission.csv', 'model']
compress(file_names)

File Paths:
['fake_news_detection.ipynb', 'submission.csv', 'model']
